<a href="https://colab.research.google.com/github/codershiva91/Email-Marketing-ML-model/blob/main/Email_Marketing_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets Import all t he necessary Libaraies

In [69]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler



LETS LOAD THE DATASETS FOR THE MODEL BUILDING

In [70]:

base_url = 'https://raw.githubusercontent.com/codershiva91/Email-Marketing-ML-model/main/'


email_df = pd.read_csv(base_url + 'email_table.csv')
opened_df = pd.read_csv(base_url + 'email_opened_table.csv')
clicked_df = pd.read_csv(base_url + 'link_clicked_table.csv')


In [71]:
email_df.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases
0,85120,short_email,personalized,2,Sunday,US,5
1,966622,long_email,personalized,12,Sunday,UK,2
2,777221,long_email,personalized,11,Wednesday,US,2
3,493711,short_email,generic,6,Monday,UK,1
4,106887,long_email,generic,14,Monday,US,6


In [72]:
opened_df.head()

,email_id
0,284534
1,609056
2,220820
3,905936
4,164034


In [73]:
clicked_df.head()

,email_id
0,609056
1,870980
2,935124
3,158501
4,177561


LETS ADD TARGETS

In [74]:
email_df["email_opened"]= email_df.email_id.isin(opened_df.email_id).astype(int)
email_df["link_clicked"]=email_df.email_id.isin(clicked_df.email_id).astype(int)


print campaign statisstics

In [75]:
open_rate=email_df.email_opened.mean()
click_rate=email_df.link_clicked.mean()

print(f"Email Open Rate: {open_rate:.2%}")
print(f"Click Through Rate: {click_rate:.2%}")

Email Open Rate: 10.35%
Click Through Rate: 2.12%


feature selection

In [76]:
features = ["email_text", "email_version", "hour", "weekday", "user_country", "user_past_purchases"]
X = email_df[features]
y = email_df["link_clicked"]


One-hot encoding

In [77]:
X = pd.get_dummies(X, columns=["email_text", "email_version", "weekday", "user_country"], drop_first=True)

lets perform mormalize numeric columns

In [78]:
# Normalize numeric columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X[["hour", "user_past_purchases"]] = scaler.fit_transform(X[["hour", "user_past_purchases"]])


now spliting in taining and testing phase

In [79]:
from sklearn.model_selection import train_test_split
X_train ,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

lets train model

In [80]:
model = RandomForestClassifier(n_estimators=100, random_state=42) # Corrected parameter name to 'n_estimators'
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [81]:
# Evaluate
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     19547
           1       0.08      0.01      0.02       453

    accuracy                           0.97     20000
   macro avg       0.53      0.50      0.50     20000
weighted avg       0.96      0.97      0.97     20000

ROC AUC: 0.5853362321030502


In [82]:
# Predicted uplift CTR
predicted_ctr = y_proba.mean()
uplift = predicted_ctr - click_rate
print(f"Predicted CTR: {predicted_ctr:.2%}")
print(f"Estimated uplift: {uplift:.2%}")


Predicted CTR: 2.10%
Estimated uplift: -0.02%


lets save the trained model

In [83]:
import joblib
joblib.dump(model, "click_prediction_model.pkl")


['click_prediction_model.pkl']

lets build a rest api
Use FastAPI (lightweight and production-friendly) to wrap your model